In [2]:
import re
import json
import numpy as np
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForTokenClassification

device = "cuda"

d:\intern\phi\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the code for generetion raw unlabeled text. 

In results generating already labeled data from copilot is a lot faster, but have worse label accuracy

In [22]:
# Load a pre-trained causal language model and tokenizer, then set up a text generation pipeline.
phi_path = 'Phi-3.5-mini-instruct'

model = AutoModelForCausalLM.from_pretrained(
    phi_path, 
    device_map=device, 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(phi_path)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

messages = [
    {"role": "system", "content": "You are a helpful AI assistant"},
    {"role": "user", "content": "Write a ten short sentences with mountain name, use different names"},

]

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 2.0,
    "do_sample": True,
}

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.50s/it]


In [ ]:
# create 1000 sentence
for  i in range(100):
    try:
        output = pipe(messages, **generation_args)

        with open('dataset.txt', 'a', encoding='utf-8') as file:
                file.write(output[0]['generated_text'] + '\n')  # Write text and move to the next line
    except:
        pass

In [ ]:
# clearing bad sentences
def clear_dataset(filename):

    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Filter out empty lines, lines that dont end with dot or longer than 150. Also clear number dot before every sentence
    c_dataset = [re.sub(r'^\d+\.\s*', '', line) for line in lines if line.rstrip().endswith('.') and len(line) <= 150]

    return c_dataset
# save it
with open('dataset.txt', 'w', encoding='utf-8') as file:
    file.write(clear_dataset('dataset.txt'))


Next you need to lable this data, i did this with Microsoft copilot
 
Next is creating dataset wrom raw text in format "word (entity) word (entity)..."

In [3]:
label_list = {'(O)': 0, '(B-PER)': 1, '(I-PER)': 2, '(B-ORG)': 3, '(I-ORG)': 4, '(B-LOC)': 5, '(I-LOC)': 6, '(B-MISC)': 7, '(I-MISC)': 8,'(B-MOU)':9,'(I-MOU)':10}

with open('BIO_dataset.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# 
id = [] # Line number
tags = [] # Entitys
tokens = [] # Words


for idx,sentence in enumerate(lines): # fill lists with data

    words = sentence.split()  # Split the sentence into words

    token_list = [] 
    tag_list = []

    for i in range(len(words)):
        # checking the format, if some word dont have corresponded entity it take -100, this number will be ignored later
        if '(' not in words[i] and ')' not in words[i]:
            if i == len(words)-1 or'(' not in words[i+1] or ')' not in words[i+1]: 
                token_list.append(words[i])
                tag_list.append(-100)
            elif '(' in words[i+1] and ')' in words[i+1]:
                token_list.append(words[i])
                tag_list.append(label_list[words[i+1]])
            
    id.append(idx)
    tags.append(tag_list)
    tokens.append(token_list)

# Create a dictionary
dataset_dict = {
    'id': id,
    'tokens': tokens,
    'ner_tags': tags
}


dataset = Dataset.from_dict(dataset_dict) # create dataset

dataset.save_to_disk('./ner_dataset') #save it

# Check the dataset format
print(dataset['id'])
print(dataset['tokens'])
print(dataset['ner_tags'])


Saving the dataset (1/1 shards): 100%|██████████| 216/216 [00:00<00:00, 53949.24 examples/s]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215]
[['Mount', 'Everest', 'stands

Dont do this

Next is unfinished idea to label data with exsistng model and manualy change B-LOC and I-LOC to B-MOU and I-MOU

its too time consuming and abandoned by now

In [7]:
# load distilbert-NER model
with open('dataset.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
Ner_local_path = "distilbert-NER"#global_path = "dslim/distilbert-NER"
distilbert_tokenizer = AutoTokenizer.from_pretrained(Ner_local_path)
distilbert_model = AutoModelForTokenClassification.from_pretrained(Ner_local_path)

nlp = pipeline("ner", model=distilbert_model, tokenizer=distilbert_tokenizer, device=device)

In [11]:
# functions to merge tokens into words from distilbert-NER output
def merge_entities(entities):
    merged = []
    i = 0

    while i < len(entities):
        current = entities[i]
        
        # Check for merging conditions
        while (i + 1 < len(entities) and 
               current['end'] == entities[i + 1]['start']):
            # Merge words
            current['word'] += entities[i + 1]['word'].replace('##', '')
            # Update the end index
            current['end'] = entities[i + 1]['end']
            # Move to the next entity
            i += 1
        
        # Add merged entity to result
        merged.append(current)
        # Move to the next entity
        i += 1

    return merged

In [ ]:
# put dataset from llm to NER model
ner_results = nlp(lines)

# Custom function to convert float32 to float
def convert_floats(obj):
    if isinstance(obj, np.float32):  # Convert float32 to float
        return float(obj)
    if isinstance(obj, np.ndarray):  # Handle NumPy arrays
        return obj.tolist()
    return obj

filtered_results= []
# filter  output from  NER model
for n in range(len(lines)):
    filtered_results.append([{key: value for key, value in result.items() if key != 'score' and key != 'index'} for result in ner_results[n]])
# make words from output tokens
for i in range(len(filtered_results)):
    filtered_results[i] = merge_entities(filtered_results[i])
# # Open the JSON file in append mode and write each result
with open("data_lables.json", 'a') as json_file:
    json.dump(filtered_results, json_file, default=convert_floats, indent=4)  
    json_file.write("\n")